In [27]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops
import torch.nn as nn

num_bond_type = 7 
num_bond_direction = 3 

class GINConv(MessagePassing):
    def __init__(self, emb_dim, aggr="add"):
        super(GINConv, self).__init__()
        self.mlp = nn.Sequential(nn.Linear(emb_dim, 2 * emb_dim), nn.ReLU(), nn.Linear(2 * emb_dim, emb_dim))
        self.edge_embedding1 = nn.Embedding(num_bond_type, emb_dim)
        self.edge_embedding2 = nn.Embedding(num_bond_direction, emb_dim)

        nn.init.xavier_uniform_(self.edge_embedding1.weight.data)
        nn.init.xavier_uniform_(self.edge_embedding2.weight.data)
        self.aggr = aggr

    def forward(self, x, edge_index, edge_attr):
        edge_index = add_self_loops(edge_index, num_nodes=x.size(0))

        self_loop_attr = torch.zeros(x.size(0), 2)
        self_loop_attr[:, 0] = 4  # bond type for self-loop edge
        self_loop_attr = self_loop_attr.to(edge_attr.device).to(edge_attr.dtype)
        edge_attr = torch.cat((edge_attr, self_loop_attr), dim=0)

        edge_embeddings = self.edge_embedding1(edge_attr[:, 0]) + self.edge_embedding2(edge_attr[:, 1])
        print(edge_attr[:, 0].shape, edge_attr[:, 1].shape)
        print(self.edge_embedding1, self.edge_embedding2)
        print(f"Shape of edge embeddings: {edge_embeddings.shape}")  # Tracking edge embeddings shape
        print(f"Shape of edge embedding 0: {self.edge_embedding1(edge_attr[:, 0])}, Shape of edge embedding 1: {self.edge_embedding2(edge_attr[:, 1])}")  # Tracking edge embedding shapes
        return self.propagate(edge_index[0], x=x, edge_attr=edge_embeddings)

    def message(self, x_j, edge_attr):
        print(f"Shape of x_j in message: {x_j.shape}, Shape of edge_attr in message: {edge_attr.shape}")  # Tracking shapes in message
        return x_j + edge_attr

    def update(self, aggr_out):
        print(f"Shape of aggr_out in update: {aggr_out.shape}")  # Tracking shape in update
        return self.mlp(aggr_out)



In [28]:

# Example usage
emb_dim = 5
gin_conv = GINConv(emb_dim=emb_dim)

# Dummy data for testing
num_nodes = 3
x = torch.randn(num_nodes, emb_dim)  # Node features
edge_index = torch.tensor([[0, 1, 2, 0], [1, 2, 0, 2]])  # Edges
edge_attr = torch.tensor([[1, 0], [2, 1], [3, 2], [0, 1]])  # Edge attributes

# Forward pass
updated_node_features = gin_conv(x, edge_index, edge_attr)
print("Updated node features:")
print(updated_node_features)


torch.Size([7]) torch.Size([7])
Embedding(7, 5) Embedding(3, 5)
Shape of edge embeddings: torch.Size([7, 5])
Shape of edge embedding 0: tensor([[-0.1417, -0.5789, -0.1907, -0.0125,  0.2462],
        [ 0.2160,  0.0211,  0.1449, -0.6272, -0.1873],
        [-0.1180, -0.5317, -0.5951, -0.5017,  0.1155],
        [-0.5059, -0.2013,  0.0832, -0.2865, -0.5461],
        [ 0.5345,  0.3491,  0.3122,  0.1297,  0.1854],
        [ 0.5345,  0.3491,  0.3122,  0.1297,  0.1854],
        [ 0.5345,  0.3491,  0.3122,  0.1297,  0.1854]],
       grad_fn=<EmbeddingBackward0>), Shape of edge embedding 1: tensor([[-0.4809,  0.7644, -0.5652,  0.3381, -0.8205],
        [ 0.1137,  0.2010,  0.2618, -0.8248, -0.7822],
        [ 0.4874, -0.6461, -0.6930, -0.8118,  0.1054],
        [ 0.1137,  0.2010,  0.2618, -0.8248, -0.7822],
        [-0.4809,  0.7644, -0.5652,  0.3381, -0.8205],
        [-0.4809,  0.7644, -0.5652,  0.3381, -0.8205],
        [-0.4809,  0.7644, -0.5652,  0.3381, -0.8205]],
       grad_fn=<EmbeddingBa

In [85]:
import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.out_channels = out_channels
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.empty(out_channels))
        self.embeddings = torch.nn.Embedding(2, out_channels)
        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index, edge_attr):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        # print(f"1. shape endge index {edge_index.shape}")
        #add features corresponding to self-loop edges.
        self_loop_attr = torch.zeros(x.size(0), 19)
        self_loop_attr[:,-1] = 1 #bond type for self-loop edge
        self_loop_attr = self_loop_attr.to(edge_attr.device).to(edge_attr.dtype)
        edge_attr = torch.cat((edge_attr, self_loop_attr), dim = 0)
        
        summed_embeddings = torch.zeros(edge_attr.size(0), self.out_channels)

        for i in range(19):  # Iterate over the second dimension
            summed_embeddings += self.embeddings(edge_attr[:, i])
        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        # print(f"2. shape x {x.shape}")
        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        # print(f"3. shape norm {norm.shape}")
        # Step 4-5: Start propagating messages.
        out = self.propagate(edge_index, x=x, edge_attr = summed_embeddings,norm=norm)
        # print(f"7. shape out {out.shape}")
        # Step 6: Apply a final bias vector.
        out = out + self.bias

        return out

    def message(self, x_j, edge_attr, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        # print(f"4. shape xj {x_j.shape}")
        # print(f"5. shape norm.vier(-1,1) {norm.view(-1, 1).shape}")
        # print(f"6. shape output {(norm.view(-1, 1) * x_j).shape}")
        return norm.view(-1, 1) * (x_j + edge_attr)

In [87]:
import torch
from torch_geometric.data import Data

# Initialize the GCNConv layer
# Transform from 3-dimensional features to 2-dimensional features
gcn_conv = GCNConv(in_channels=3, out_channels=2)

# Define node features (4 nodes with 3 features each)
x = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]], dtype=torch.float)

# Define the edges in the graph (making it undirected)
# Each pair of nodes is connected in both directions
edge_index = torch.tensor([[0, 1, 1, 2, 2, 3, 3, 0, 0, 2], 
                           [1, 0, 2, 1, 3, 2, 0, 3, 2, 0]], dtype=torch.long)  # Edges: 0-1, 1-2, 2-3, 3-0, 0-2
edge_attr = torch.randint(0, 1, (edge_index.size(1), 19))
# Apply the GCNConv layer to the node features
out_features = gcn_conv(x, edge_index, edge_attr)

print("Original node features:\n", x)
print("\nTransformed node features:\n", out_features)


Original node features:
 tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.],
        [10., 11., 12.]])

Transformed node features:
 tensor([[ 10.4818, -20.3537],
        [  9.5020, -15.6413],
        [ 10.4818, -20.3537],
        [  8.4351, -17.7163]], grad_fn=<AddBackward0>)


In [80]:
from torch_scatter import scatter_add
class GCNConv(MessagePassing):
    
    def __init__(self, in_channels, out_channels, aggr = "add"):
        super(GCNConv, self).__init__()

        self.out_channels = out_channels
        self.linear = torch.nn.Linear(in_channels, out_channels)
        self.edge_embedding = torch.nn.Embedding(2, out_channels)

        torch.nn.init.xavier_uniform_(self.edge_embedding.weight.data)

        self.aggr = aggr

    def norm(self, edge_index, num_nodes, dtype):
        ### assuming that self-loops have been already added in edge_index
        edge_weight = torch.ones((edge_index.size(1), ), dtype=dtype,
                                     device=edge_index.device)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        return deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]


    def forward(self, x, edge_index, edge_attr):
        #add self loops in the edge space
        edge_index, _ = add_self_loops(edge_index, num_nodes = x.size(0))

        #add features corresponding to self-loop edges.
        self_loop_attr = torch.zeros(x.size(0), 19)
        self_loop_attr[:,-1] = 1 #bond type for self-loop edge
        self_loop_attr = self_loop_attr.to(edge_attr.device).to(edge_attr.dtype)
        edge_attr = torch.cat((edge_attr, self_loop_attr), dim = 0)

        edge_embeddings = torch.zeros(edge_attr.size(0), self.out_channels)
        for i in range(19):  # Iterate over the second dimension
            edge_embeddings += self.edge_embedding(edge_attr[:, i])

        norm = self.norm(edge_index, x.size(0), x.dtype)

        x = self.linear(x)

        return self.propagate(edge_index, x=x, edge_attr=edge_embeddings, norm=norm)

    def message(self, x_j, edge_attr, norm):
        return norm.view(-1, 1) * (x_j + edge_attr)

In [81]:
in_channels = 42
out_channels = 512  # Should match the second dimension of edge_attr
x = torch.randint(0, 1, (4, in_channels))  # Node features [4, 4on-singleton dimension 0
edge_attr = torch.randint(0, 1, (6, 19))  # Edge attributes [6, 19]

gcn_conv = GCNConv(in_channels, out_channels)
out_features = gcn_conv(x, edge_index, edge_attr)

print("Output node features:")
print(out_features)

RuntimeError: mat1 and mat2 must have the same dtype

In [71]:
import torch
import torch.nn as nn

# Simulating input tensor of shape [14, 19] with indices
# Assuming each entry is an index, they should be integers (long type)
# For simplicity, we're generating random indices here. In practice, these should be your actual data.
input_tensor = torch.randint(0, 512, (14, 19), dtype=torch.long)

# Create an embedding layer
# Assuming maximum index is 511, thus embedding size is 512
# Each index will map to a 512-dimensional vector
embedding = nn.Embedding(512, 512)

# Since Embedding expects a 1D input to map to a 1D output, we need to handle each "feature" separately
# We can do this by iterating over the second dimension and summing or concatenating the embeddings
# Here, we sum them for demonstration purposes

# Initialize a tensor to store the sum of embeddings
summed_embeddings = torch.zeros(14, 512)

for i in range(19):  # Iterate over the second dimension
    summed_embeddings += embedding(input_tensor[:, i])

print("Summed embeddings shape:", summed_embeddings.shape)

Summed embeddings shape: torch.Size([14, 512])
